# Explore here

In [3]:
!kaggle competitions download -c DontGetKicked

 49%|██████████████████▌                   | 5.00M/10.2M [00:00<00:00, 36.8MB/s]
100%|██████████████████████████████████████| 10.2M/10.2M [00:00<00:00, 66.3MB/s]


In [4]:
import zipfile

# Define the path to your zip file
zip_path = '/workspaces/Isabell-Joane-Eric-Final-Project/src/DontGetKicked.zip'

# Define the directory where you want to extract the contents
extract_to = '/workspaces/Isabell-Joane-Eric-Final-Project/data/processed'

# Open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the contents into the directory
    zip_ref.extractall(extract_to)


In [5]:
import pandas as pd

# Path to the CSV file
file_path = '/workspaces/Isabell-Joane-Eric-Final-Project/data/processed/training.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to confirm it's loaded correctly
print(df.head())

   RefId  IsBadBuy  PurchDate Auction  VehYear  VehicleAge   Make  \
0      1         0  12/7/2009   ADESA     2006           3  MAZDA   
1      2         0  12/7/2009   ADESA     2004           5  DODGE   
2      3         0  12/7/2009   ADESA     2005           4  DODGE   
3      4         0  12/7/2009   ADESA     2004           5  DODGE   
4      5         0  12/7/2009   ADESA     2005           4   FORD   

                 Model Trim           SubModel  ...  \
0               MAZDA3    i         4D SEDAN I  ...   
1  1500 RAM PICKUP 2WD   ST  QUAD CAB 4.7L SLT  ...   
2           STRATUS V6  SXT   4D SEDAN SXT FFV  ...   
3                 NEON  SXT           4D SEDAN  ...   
4                FOCUS  ZX3       2D COUPE ZX3  ...   

  MMRCurrentRetailAveragePrice MMRCurrentRetailCleanPrice  PRIMEUNIT AUCGUART  \
0                      11597.0                    12409.0        NaN      NaN   
1                      11374.0                    12791.0        NaN      NaN   
2          

In [10]:
df.shape

(72983, 34)